In [1]:
#hide
#default_exp docs

# nbprocess.docs
- Exporting a notebook to docs

In [2]:
#export
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.export import *
from nbprocess.sync import write_nb, nb2dict

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

import uuid
import tempfile

In [3]:
#hide
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

from nbconvert import MarkdownExporter
import traitlets.config

__file__ = '../nbprocess/export.py'

In [4]:
#export
def rm_blank_proc(cell):
    "Remove empty cells"
    if(cell.source.strip()==''): cell.source = None

In [5]:
#export
_re_ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

def strip_ansi_proc(cell):
    "Strip Ansi Characters"
    for o in cell.get('outputs', []):
        if o.get('name') == 'stdout': o['text'] = _re_ansi_escape.sub('', o.text)

In [26]:
#export
def html_escape(cell):
    "Place HTML in a codeblock and surround it with a <HTMLOutputBlock> component."
    for o in cell.get('outputs', []):
        html = nested_idx(o, 'data', 'text/html')
        if html:
            cell.metadata.html_output = True
            html = ''.join(html).strip()
            o['data']['text/html'] = f'```html\n{html}\n```'

In [27]:
#export
def _get_cell_id(id_length=36): return uuid.uuid4().hex[:id_length]

def _get_md_cell(content="<!--- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!-->"):
    return AttrDict({'cell_type': 'markdown', 'id': f'{_get_cell_id()}',
                     'metadata': {}, 'source': f'{content}'})

def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    nb.cells = nb.cells[:1] + [_get_md_cell()] + nb.cells[1:]

In [28]:
#     c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell", "hide")
#     c.TagRemovePreprocessor.remove_all_outputs_tags = ("remove_output", "remove_outputs", "hide_output", "hide_outputs")
#     c.TagRemovePreprocessor.remove_input_tags = ('remove_input', 'remove_inputs', "hide_input", "hide_inputs")
#     pp = [InjectMeta, WriteTitle, CleanMagics, BashIdentify, MetaflowTruncate,
#           MetaflowSelectSteps, UpdateTags, TagRemovePreprocessor, CleanFlags, CleanShowDoc, RmEmptyCode,
#           HideInputLines, Black, ImageSave, ImagePath, HTMLEscape]
#     c.MarkdownExporter.preprocessors = pp

In [29]:
#export
def write_md(nb_path, procs=None, post_procs=None, tpl_file='ob.tpl'):
    nbp = NBProcessor(nb_path, procs)
    nb = nbp.nb
    nbp.process()
    for proc in L(post_procs): proc(nb)

    c = traitlets.config.Config()
    base = Path(__file__).parent.resolve()
    c.MarkdownExporter.template_file = str(base/'templates'/tpl_file)
    exp = MarkdownExporter(config=c)

    with tempfile.TemporaryFile('a+') as tmp:
        write_nb(nb, tmp)
        tmp.seek(0)
        md,_ = exp.from_file(tmp)

    dest = base/'../docusaurus/docs'/Path(nb_path).with_suffix('.md').name
    dest.write_text(md)
    return dest

In [34]:
procs = [rm_blank_proc, strip_ansi_proc, html_escape]
post_procs = [insert_warning]

path = '../tests/docs_test.ipynb'
write_md(path, procs, post_procs)

Path('/home/jhoward/git/nbprocess/nbprocess/../docusaurus/docs/docs_test.md')

In [31]:
!cat ../docusaurus/docs/docs_test.md

## a title

<!--- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!-->

some md


```python
import re
```


```python
print('\033hello')
```

<CodeOutputBlock lang="python">

    hello


</CodeOutputBlock>


```python
1+1
```

<CodeOutputBlock lang="python">




    2



</CodeOutputBlock>


```python
%%html
<b>a test</b>
```
    
<HTMLOutputBlock >


```html
<b>a test</b>
```


</HTMLOutputBlock>


```python
import pandas as pd
print('hi')
pd.DataFrame(dict(a=[1,2]))
```
    
<HTMLOutputBlock >

    hi





```html
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>a</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>

## Export -

In [33]:
#skip
from nbprocess.export import nbs_export
nbs_export()

In [4]:
#     res = ''
#     for cell in nb.cells:
#         src = cell.source
#         if cell.cell_type=='code': src = f"\n```{lang}\n{src}\n```\n"
#         res += src + '\n'
#         if 'outputs' in cell:
#             for outp in cell.outputs:
#                 d = outp.get('data', {})
#                 if 'text/html' in d:
#                     res += '\n'.join(d['text/html'])
#                 elif 'text/plain' in d:
#                     t = '\n'.join(d['text/plain'])
#                     res += f"```\n  {t}\n```\n"
#     return res